In [1]:
import findspark
findspark.init()
findspark.find()
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import pyspark
from pyspark.sql import SparkSession

import datetime
import pyspark.sql.functions as F 
import dateutil.relativedelta

spark = SparkSession.builder \
            .master("yarn") \
            .appName("akk_2") \
            .config("spark.executor.memory", "5530m")\
            .config("spark.executor.instances", 4)\
            .config("spark.executor.cores", 2)\
            .config("spark.files.overwrite",True)\
            .getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-01-10 09:35:52,019 WARN util.Utils: Your hostname, fhmg3ps3ao3or6m6jajr resolves to a loopback address: 127.0.1.1; using 172.16.0.20 instead (on interface eth0)
2023-01-10 09:35:52,020 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-01-10 09:36:06,652 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [2]:
tags_verified = spark.read.parquet('/user/master/data/snapshots/tags_verified/actual')

In [3]:


# def input_paths(date,depth):
#     range_ = []
#     date = datetime.datetime.strptime(date, "%Y-%m-%d")
#     i = 0
#     while i < depth:
#         date_2 = date - dateutil.relativedelta.relativedelta(days=i)
#         path_ = f'/user/andrew_0/data/events/date={date_2.strftime("%Y-%m-%d")}/event_type=message'
#         range_.append(path_)
#         i = i+1
#     return(range_)


# range_ = input_paths(date_from,depth)

# df = spark.read.parquet(*range_)

# df_explode = df.withColumn("tag", F.explode(df.event.tags)).where("event.message_channel_to is not null")
# df_explode = df_explode.select(df_explode.event.message_from.alias("message_from"),df_explode.tag)

# df_explode = df_explode.groupBy('tag') \
#                       .agg(F.countDistinct('message_from').alias("suggested_count")) \
#                       .filter(F.col('suggested_count') >= 100) \
#                       .orderBy(F.col("suggested_count").desc()) \
#                       .join(tags_verified, ['tag'], how='left_anti')

# df_explode.write.format('parquet').save('/user/andrew0/data/analytics/candidates_d84_pyspark',mode='overwrite')

# df_7 = spark.read.parquet('/user/andrew0/data/analytics/candidates_d7_pyspark')
# df_7.describe().show()

# df_84 = spark.read.parquet('/user/andrew0/data/analytics/candidates_d84_pyspark')
# df_84.describe().show()



# def input_paths(date, depth):
#     dt = datetime.datetime.strptime(date, '%Y-%m-%d')
#     return [f"/user/username/data/events/date={(dt-datetime.timedelta(days=x)).strftime('%Y-%m-%d')}/event_type=message" for x in range(depth)]

# paths = input_paths('2022-05-31', M)
# messages = spark.read.parquet(*paths)
# all_tags = messages.where("event.message_channel_to is not null").selectExpr(["event.message_from as user", "explode(event.tags) as tag"]).groupBy("tag").agg(F.expr("count(distinct user) as suggested_count")).where("suggested_count >= 100")
# verified_tags = spark.read.parquet("/user/master/data/snapshots/tags_verified/actual")
# candidates = all_tags.join(verified_tags, "tag", "left_anti")

# candidates.write.parquet(f'/user/username/data/analytics/candidates_d{M}_pyspark')

# /usr/lib/spark/bin/spark-submit --master yarn --deploy-mode cluster verified_tags_candidates.py 2022-05-31 5 300 

# /user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/date=2022-05-31

# date_from = '2022-05-31'
# depth = 5
# sugested_users = 300
# input_path = '/user/andrew_0/data/events'
# tags_verified_path = '/user/master/data/snapshots/tags_verified/actual'
# output_path = '/user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/'+'date='+date_from

 



In [66]:
interests_d7_eth =  spark.read.parquet('/user/examples/data/interests_d7')
interests_d7_eth.show(10,False)
# interests_d28_eth = spark.read.parquet('/user/examples/data/interests_d28')
# interests_d28_eth.show(10,False)

Py4JJavaError: An error occurred while calling o1328.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=andrew0, access=READ_EXECUTE, inode="/user/examples/data/interests_d7":ubuntu:hadoop:d-wx--x--x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:399)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:261)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:193)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1896)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1880)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPathAccess(FSDirectory.java:1830)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getListingInt(FSDirStatAndListingOp.java:78)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.

In [55]:
date_from = '2022-06-04'
depth = 5
sugested_users = 300
input_path = '/user/andrew_0/data/events'
tags_verified_path = '/user/master/data/snapshots/tags_verified/actual'
output_path = '/user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/'+'date='+date_from

In [56]:
# range_

In [57]:
def input_paths(date,depth):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type=message'
        range_.append(path_)
        i = i+1
    return(range_)

In [61]:
range_ = input_paths(date_from,depth)

df = spark.read.parquet(*range_)


In [59]:
from pyspark.sql.window import Window

window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())

In [62]:
def tag_tops(date_from,depth,spark):
    
    range_ = input_paths(date_from,depth)
    
    df = spark.read.parquet(*range_)
    
    window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())
    
    
    df = df.where("event.message_channel_to is not null")\
    .select([F.col("event.message_from").alias("user"), F.explode(F.col("event.tags")).alias("tag")])\
    .groupBy(["user","tag"]).agg(F.count("tag").alias("suggested_count"))\
    .withColumn("r_n",F.row_number().over(window_))\
    .withColumn("tag_top_1",F.lead("tag",0).over(window_))\
    .withColumn("tag_top_2",F.lead("tag",1).over(window_))\
    .withColumn("tag_top_3",F.lead("tag",2).over(window_))\
    .where("r_n = 1")\
    .select([F.col('user').alias('user_id'),F.col('tag_top_1'),F.col('tag_top_2'),F.col('tag_top_3')])
    
    print(date_from,depth)
    return df

In [63]:
tag_tops('2022-06-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5')

tag_tops('2022-05-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_5')

tag_tops('2022-05-04', 1, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_1')

2022-06-04 5


2022-05-04 5


2022-05-04 1


In [65]:
spark.read.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5').show(10,False)

+-------+-------------------------------+--------------+-----------+
|user_id|tag_top_1                      |tag_top_2     |tag_top_3  |
+-------+-------------------------------+--------------+-----------+
|77947  |Старое железо                  |Краудсорсинг  |История IT |
|87917  |heaps                          |denmark       |null       |
|91431  |Чулан                          |hardware-alley|disrupt-nyc|
|33851  |Разработка под Android         |nielsen       |apps       |
|3302   |rapportive                     |mailbox       |breather   |
|81061  |local-advertising              |null          |null       |
|82347  |Чулан                          |Браузеры      |xbox-one   |
|86802  |Разработка мобильных приложений|ReactJS       |IPv6       |
|122239 |social-shopping                |shopinterest  |pintics    |
|143136 |voip                           |recipes       |food52     |
+-------+-------------------------------+--------------+-----------+
only showing top 10 rows

